<h1><center>Laboratorio 10: Persistencia, APIs y Servidores 💦</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Pablo Badilla
- Auxiliar: Ignacio Meza D.
- Ayudante: Patricio Ortiz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Tinoco
- Nombre de alumno 2:

### **Link de repositorio de GitHub:** `https://github.com/sebatinoco/MDS7202/tree/main/lab10`

# Temas a tratar

- Persistencia de modelos.
- APIs Rest.
- Servidores.
- Bases de Datos basadas en JSON (formato similar a diccionarios).

## Reglas:

- Fecha de entrega: 7/07/2021
- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias. 
- Pueden usar cualquer matrial del curso que estimen conveniente.

### Objetivos principales del laboratorio

- Utilizar `scikit-learn` para generar un modelo y luego guardarlo a un achivo.
- Entender qué es una API y qué significa que esta cumpla con los principios de REST.
- Utilizar `FastAPI` para crear un servicio que permita usar el modelo.
- Guardar llamadas a la API y obtener sus resultados usando una base de datos basada en documentos.


## Introducción: Crear y Persistir un Modelo

<div align='center'/>
  <img src="https://pbs.twimg.com/media/E1jfjTRWYAAfi2v.jpg" width="500">
</div>

Consumida en la tristeza el despido de Renacín, Smapina ha decaído en su desempeño, lo que se ha traducido en un irregular tratamiento del agua. Esto ha implicado una baja en la calidad del agua, llegando a haber algunos puntos de la comuna en la que el vital elemento no es apto para el consumo humano. Es por esto que la sanitaria pública de la municipalidad de Maipú se ha contactado con ustedes para que le entreguen una urgente solución a este problema (a la vez que dejan a Smapina, al igual que Renacín, sin trabajo 😔). 

El problema que la empresa le ha solicitado resolver es el de elaborar un sistema que les permita saber si el agua es potable o no. Para esto, la sanitaria les ha proveido una base de datos con la lectura de múltiples sensores IOT colocados en diversas cañerías, conductos y estanques. Estos sensores señalan nueve tipos de mediciones químicas y más una etiqueta elaborada en laboratorio que indica si el agua es potable o no el agua.

La idea final es que puedan, en el caso que el agua no sea potable, dar un aviso inmediato para corregir el problema. Tenga en cuenta que parte del equipo docente vive en Maipú y su intoxicación podría implicar graves problemas para el cierre del curso.

Atributos:

1. pH value
2. Hardness
3. Solids (Total dissolved solids - TDS)
4. Chloramines
5. Sulfate
6. Conductivity
7. Organic_carbon
8. Trihalomethanes
9. Turbidity

Variable a predecir: 

10. Potability (1 si es potable, 0 no potable)

\[Fuera de rol\] Los datos originales, junto a la descripción de cada atributo se encuentran en: 

https://www.kaggle.com/adityakadiwal/water-potability

## Modelo a Utilizar

<div align="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/6/6c/SMAPA.jpg" width="400">
</div>

Al recibir el trabajo, se dan cuenta que el equipo de TI de la sanitaria, de forma autodidacta, ya estaba trabajando en un proyecto de predicción de agua potable utilizando *Machine Learning*. Sí bien el código no es el mejor, ustedes como equipo consideran que el trabajo de predicción ya está realizado y prefieren utilizar todos sus recursos en la ardua tarea de desplegarlo.

Por lo mismo, lea y ejecute el código proveido para entrenar y guardar el modelo y luego continue con el despliegue del sistema (No debe hacer nada en esta parte, solo entender el problema que se quiere resolver).

In [10]:
import numpy as np
import pandas as pd
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import classification_report
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [2]:
try:
    from google.colab import drive
    # ejecutar en el caso de usar colab.
    drive.mount("/content/drive")
except:
    pass

In [3]:
df = pd.read_csv("./water_potability.csv").dropna()
df

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
5,5.584087,188.313324,28748.687739,7.544869,326.678363,280.467916,8.399735,54.917862,2.559708,0
6,10.223862,248.071735,28749.716544,7.513408,393.663396,283.651634,13.789695,84.603556,2.672989,0
7,8.635849,203.361523,13672.091764,4.563009,303.309771,474.607645,12.363817,62.798309,4.401425,0
...,...,...,...,...,...,...,...,...,...,...
3267,8.989900,215.047358,15921.412018,6.297312,312.931022,390.410231,9.899115,55.069304,4.613843,1
3268,6.702547,207.321086,17246.920347,7.708117,304.510230,329.266002,16.217303,28.878601,3.442983,1
3269,11.491011,94.812545,37188.826022,9.263166,258.930600,439.893618,16.172755,41.558501,4.369264,1
3270,6.069616,186.659040,26138.780191,7.747547,345.700257,415.886955,12.067620,60.419921,3.669712,1


In [4]:
X = df.drop(columns=["Potability"])
y = df.loc[:, ["Potability"]].values.ravel()

# Holdout
X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=42, shuffle=True, stratify=y
)

# Pipeline
pipe = Pipeline(
    [
        ("Scaler", StandardScaler()),
        ("Clf", BaggingClassifier()),
    ]
)

# Grilla de Parámetros
param_grid = {
    "Clf__max_features": [0.5, 0.7, 1.0],
    "Clf__max_samples": [0.5, 0.7, 1.0],
    "Clf__n_estimators": [5, 10, 15, 20, 25, 50],
}

# Buscar la mejor configuración de parámetros usando HalvingGridSearchCV
gs = HalvingGridSearchCV(pipe, param_grid, verbose=100, cv=5, n_jobs=-1)
gs.fit(X_train, y_train)

gs


n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 55
max_resources_: 1508
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 54
n_resources: 55
Fitting 5 folds for each of 54 candidates, totalling 270 fits
----------
iter: 1
n_candidates: 18
n_resources: 165
Fitting 5 folds for each of 18 candidates, totalling 90 fits
----------
iter: 2
n_candidates: 6
n_resources: 495
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 1485
Fitting 5 folds for each of 2 candidates, totalling 10 fits


HalvingGridSearchCV(estimator=Pipeline(steps=[('Scaler', StandardScaler()),
                                              ('Clf', BaggingClassifier())]),
                    n_jobs=-1,
                    param_grid={'Clf__max_features': [0.5, 0.7, 1.0],
                                'Clf__max_samples': [0.5, 0.7, 1.0],
                                'Clf__n_estimators': [5, 10, 15, 20, 25, 50]},
                    verbose=100)

In [5]:
print(f"Mejor Modelo Grid Search: {gs.best_estimator_}\n")
print(f"Mejores Parámetros: {gs.best_params_}\n")

print(f"Mejor Accuracy Grid Search: {gs.best_score_}\n")

best_random_forest = gs.best_estimator_
y_pred = best_random_forest.predict(X_test)

print("Reporte de Clasificación\n")
print(classification_report(y_test, y_pred))

Mejor Modelo Grid Search: Pipeline(steps=[('Scaler', StandardScaler()),
                ('Clf',
                 BaggingClassifier(max_features=0.7, max_samples=0.5,
                                   n_estimators=20))])

Mejores Parámetros: {'Clf__max_features': 0.7, 'Clf__max_samples': 0.5, 'Clf__n_estimators': 20}

Mejor Accuracy Grid Search: 0.6122622622622622

Reporte de Clasificación

              precision    recall  f1-score   support

           0       0.67      0.88      0.76       300
           1       0.67      0.35      0.46       203

    accuracy                           0.67       503
   macro avg       0.67      0.62      0.61       503
weighted avg       0.67      0.67      0.64       503



Y use `dump` de `joblib` para persistir el modelo recién entrenado en el disco.

In [6]:
from joblib import dump, load

dump(gs, "modelo.joblib")

['modelo.joblib']

## Parte 1: Conceptos \[3 Puntos\]

<div align="center">
  <img src="https://media1.tenor.com/images/14c1bcb6e0eb7c5ff4ac225f1114e819/tenor.gif?itemid=13036231" width="400">
</div>

Antes de comenzar a trabajar, la administración de la empresa de agua les pide explicar la ideas claves para el desarrollo de una aplicación web. 

Para esto, les pide explicar muy brevemente los siguientes conceptos:

**Nota:** Puede ser de ayuda revisar la clase 24. 

> ¿Qué es el modelo Cliente-Servidor?

La arquitectura cliente-servidor es un modelo de diseño de software en el que las tareas se reparten entre los proveedores de recursos o servicios, llamados servidores, y los demandantes, llamados clientes. Un cliente realiza peticiones a otro programa, el servidor, quien le da respuesta. El modelo Cliente-Servidor se compone de:

- Cliente: Persona que demanda un servicio y espera una respuesta del servidor, usualmente a través de solicitudes hechas mediante API o una interfaz gráfica.

- Servidor: Al contrario del cliente, el servidor se encarga de proveer el servicio requerido, procesando las solicitudes y emitiendo una respuesta. Una particularidad es que puede procesar varias solicitudes al mismo tiempo.
    
> ¿Qué es una URI?

Similar a la URL, una URI (Uniform Resource Identifier) es un localizador de un recurso web y un protocolo que permite acceder a este. Se compone de:

- Protocolo: por ejemplo, `https`

- Dirección del recurso: por ejemplo, `en.wikipedia.org`

- Archivo: por ejemplo,`Never_Gonna_Give_You_Up` (el cual es interpretado como html)

Por último, una URI se diferencia de la URL en que permite incluir en la dirección una subdirección, determinada por el “fragmento”.
    
    
> ¿Qué es HTTP?¿Qué significa que sea stateless?

En primer lugar, un protocolo es un sistema de reglas que permite que dos o mas entidades se comuniquen con el fin de transmitir información. De esta forma, un protocolo define la sintaxis, semántica y sincronización de la comunicación.

En ese sentido, HTTP es uno de los protocolos mas usados a nivel global, permitiendo la comunicación a través de archivos html y otros formatos. HTPP usa la siguiente sintaxis:

- Cabeceras (Headers) que indican el protocolo.
- Método de petición (`GET`, `POST`, `PUT`, `DELETE`, `HEAD`, `OPTION`, etc...)
- Códigos de respuesta (`1xx`, `2xx`, `3xx`, `4xx`, `5xx`), como el 404 Not Found.
- Cuerpo del mensaje

Finalmente, que HTTP sea *stateless* significa que el servidor no necesita retener la información o status de cada interacción con los clientes. En ese sentido, el servidor "olvida" la información del cliente una vez la transacción termine. Un punto importante es que esto es distinto a las cookies, lo cual son un mecanismo para que servicios con protocolo HTTP si puedan recordar información de los usuarios.
    
> ¿En que consisten las operaciones de HTTP?: 
   
- GET
    
Es una solicitud para pedir datos del servidor. 
        
- POST
    
Es una solicitud para enviar datos al servidor.
        
- PUT
    
Es una solicitud para actualizar uno o más datos del servidor.
        
- DELETE
    
Es una solicitud para eliminar uno o más datos del servidor.
        

> ¿Qué indican los códigos de respuesta de HTTP?

- `1xx`: Respuestas informativas (como el 102 - Processing)
- `2xx`: Respuestas satisfactorias (como el 200 - OK) 
- `3xx`: Redirecciones (como el 302 - Moved Permanently)
- `4xx`: Errores de los clientes (como el 404 - Not Found)
- `5xx`: Errores de los servidores (como el 501 - Not Implemented)

> ¿Qué es un Hipermedio como HTML?

Un Hipermedio es la representación de la información recibida por el servidor mediante lenguanjes como HTML o XML. Un aspecto particular de los hipermedios es que es posible navegar a otro recurso REST simplemente siguiendo enlaces en el hipermedio recibido.

> ¿Qué es una API?

Una Interfaz de Programación de Aplicaciones o API (también conocidas como Endpoints) es el conjunto de funciones que nos permiten interactuar con el servidor. Comunmente esto se hace a través de URLs parametrizadas.

> ¿Cómo API REST engloba los términos anteriores?

Finalmente, una API REST (representational state transfer) cumple con:

- Un protocolo cliente/servidor sin estado (stateless): cada mensaje HTTP contiene toda la información necesaria para comprender la petición. Como resultado, ni el cliente ni el servidor necesitan recordar ningún estado de las comunicaciones entre mensajes.

- Un conjunto de operaciones bien definidas que se aplican a todos los recursos de información: HTTP en sí define un conjunto pequeño de operaciones, las más importantes son POST, GET, PUT y DELETE.

- Una sintaxis universal para identificar los recursos. En un sistema REST, cada recurso es direccionable únicamente a través de su URI.

- El uso de hipermedios, tanto para la información de la aplicación como para las transiciones de estado de la aplicación: la representación de este estado en un sistema REST son típicamente HTML o XML. 
         
> Indique si sería buena idea o no almacenar las observaciones y predicciones que se efectuen en el servicio de predicción de agua potable y el por qué.

Finalmente, no es recomendable almacenar datos en el mismo servicio de pedicción pues, para grandes volúmenes de datos, se requeriría hardware (por ejemplo, disco duro y RAM) capaz de almacenar las bases de datos y al mismo tiempo generar predicciones. En ese sentido, lo óptimo es dividir las tareas en servidores independientes, donde cada uno se dedique exclusivamente a una tarea en específico y así minimizar el hardware requerido. Así, si un cliente desea obtener una predicción, un servidor se ocupa especificamente de computar esta predicción, mientras el otro servidor toma la predicción generada y la guarda en una base de datos (por ejemplo, MongoDB).

---
## Parte 2: Implementación del Servidor [3 Puntos]


<div align="center">
  <img src="https://media3.giphy.com/media/YQitE4YNQNahy/giphy-downsized-large.gif" width="500">
</div>

La municipal de agua les pide variadas funcionalidades al servidor que crearán. Su única restricción es que la implementen usando `FastAPI`.

Junto a la tarea se les entrega además un archivo main.py que implementa el servidor en donde están definidas las funciones que deben implementar. Ustedes deben implementar el contenido de estas funciones.

Instrucciones iniciales:

- Para instalar el servidor: `pip install fastapi[all]`. 
- Para iniciarlo, en la consola de su archivo principal, ejecutar `uvicorn main:app --reload`. 
- El servidor se autorecarga cada vez que guardan un cambio en el código. 

> Nota: Recuerden usar las documentaciones para probar las APIs que desarrollen: http://127.0.0.1:8000/docs#/

> Nota 2: Les aconsejamos seriamente intentar usar el debuger de vscode para programar el servidor.

# **Código en main.py!**

#### 1. \[1 Punto\] Predicción con `POST`

A través de un `POST` a la ruta `/potabilidad` prediga si una medición de agua es o no potable. Una llamada a esta ruta debe además registrar los datos entregados más la fecha de la predicción y la clase predicha en una base de datos (ver la sección de `TinyDB` para mayor información).


Por ejemplo, una llamada de esta ruta con un Body: 


```json
{
   "ph":10.316400384553162,
   "Hardness":217.2668424334475,
   "Solids":10676.508475429378,
   "Chloramines":3.445514571005745,
   "Sulfate":397.7549459751925,
   "Conductivity":492.20647361771086,
   "Organic_carbon":12.812732207582542,
   "Trihalomethanes":72.28192021570328,
   "Turbidity":3.4073494284238364
}```

Su serivdor debería retornar una respuesta HTML con código 200 con: 


```json
{
  "potabilidad": 0,
  "id": 121 # el id/índice en donde este elemento fue insertado en la BBDD.
}
```

Nota: puede variar según el ejemplo y el clasificador que entrenen. 


#### 2. \[0.5 Puntos\] Obtener Datos con `Get`


A través de un `GET` a la ruta `/potabilidad/` entrege todas las mediciones guardadas.


Por ejemplo: 

http://127.0.0.1:8000/potabilidad

Debería retornar todos los registros con su fecha y predicción

```json
[
   {
      "ph":10.316400384553162,
      "Hardness":217.2668424334475,
      "Solids":10676.508475429378,
      "Chloramines":3.445514571005745,
      "Sulfate":397.7549459751925,
      "Conductivity":492.20647361771086,
      "Organic_carbon":12.812732207582542,
      "Trihalomethanes":72.28192021570328,
      "Turbidity":3.4073494284238364,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   },
   {
      "ph":6.599593818953803,
      "Hardness":198.323260816452,
      "Solids":21078.887166441476,
      "Chloramines":6.23423148205243,
      "Sulfate":331.49563679988063,
      "Conductivity":447.9877914293426,
      "Organic_carbon":12.93630432875392,
      "Trihalomethanes":59.833619304688185,
      "Turbidity":4.702310752770497,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   },
   {
      "ph":7.920794564373236,
      "Hardness":173.35715056714022,
      "Solids":13454.634777245552,
      "Chloramines":6.947895137367302,
      "Sulfate":356.768119344542,
      "Conductivity":471.3686945543212,
      "Organic_carbon":16.47783972341182,
      "Trihalomethanes":68.00050835002364,
      "Turbidity":3.3743265226271983,
      "Day":17,
      "Month":7,
      "Year":2021,
      "Prediction":0
   }, 
   ...
]
```

Nota: Esta es bastante sencilla, no debería tomarles más de 2 lineas.

#### 3. \[0.5 Puntos\] `GET` con parámetros
 A través de un `GET` a la ruta `/potabilidad_diaria/` con los parámetros día, mes y año entrege todas las mediciones guardadas correspondientes a esa fecha.

Por ejemplo: 

http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021

Debería entregar las mediciones asignadas al día 17 del mes 7 del año 2021:

```json

[
    {"ph":10.316400384553162,"Hardness":217.2668424334475,"Solids":10676.508475429378,"Chloramines":3.445514571005745,"Sulfate":397.7549459751925,"Conductivity":492.20647361771086,"Organic_carbon":12.812732207582542,"Trihalomethanes":72.28192021570328,"Turbidity":3.4073494284238364,"Day":17,"Month":7,"Year":2021,"Prediction":0},{"ph":6.599593818953803,"Hardness":198.323260816452,"Solids":21078.887166441476,"Chloramines":6.23423148205243,"Sulfate":331.49563679988063,"Conductivity":447.9877914293426,"Organic_carbon":12.93630432875392,"Trihalomethanes":59.833619304688185,"Turbidity":4.702310752770497,"Day":17,"Month":7,"Year":2021,"Prediction":0},{"ph":7.920794564373236,"Hardness":173.35715056714022,"Solids":13454.634777245552,"Chloramines":6.947895137367302,"Sulfate":356.768119344542,"Conductivity":471.3686945543212,"Organic_carbon":16.47783972341182,"Trihalomethanes":68.00050835002364,"Turbidity":3.3743265226271983,"Day":17,"Month":7,"Year":2021,"Prediction":0},
...
]
```

**NOTA**: En la BBDD de prueba existen registros para los días 14, 15, 16, 17, 18 y 19 del 7 de 2021.



#### 4. \[0.5 Puntos\] `PUT` con parámetros:

A través de un `PUT` a la ruta `/potabilidad/`, con los parámetros día, mes, año y clase nueva, actualize la potabilidad de las observaciones según lo entregado.

Ejemplo: 

http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021&new_prediction=1

Debería actualizar la predicción de todos los registros del día 17/07/2021 a la clase 1 y retornar si fue exitoso o no y una lista de los documentos actualizados.


```json
{
  "success": true,
  "updated_elements": [
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80
  ]
}
```

#### 5. \[0.5 Puntos\] `DELETE` con parámetros:

A través de un `DELETE` a la ruta `/potabilidad/`, con parámetros día, mes y año, elimine todas las observaciones de ese día.

Ejemplo: 
    
http://127.0.0.1:8000/potabilidad/?day=17&month=7&year=2021
    
Debería retornar: 
    
    
```json
{
  "success": true,
  "deleted_elements": [
    61,
    62,
    63,
    64,
    65,
    66,
    67,
    68,
    69,
    70,
    71,
    72,
    73,
    74,
    75,
    76,
    77,
    78,
    79,
    80
  ]
}

```

----

### Paréntesis: Tutorial BBDD y `TinyDB`


Un sistema gestor de base de datos (SGBD) según wikipedia es: 
 
> Un conjunto de programas que permiten el almacenamiento, modificación y extracción de la información en una base de datos.

En general, es la interfaz con la cual nos comunicamos con la BBDD y que a la vez, administra los datos y todo lo relacionado con estos.

---

`TinyDB` es un gestor de base de datos minimalista para Python orientada a documentos. Es decir, está enfocada a guardar datos en forma de diccionarios. En nuestro caso, la usaremos como ejemplo para el lab, pero en el caso general, es recomendable usar mejores gestores de BBDD de documentos como MongoDB (la cual, se parece mucho a esta).


A continuación un ejemplo de uso (que pueden usar junto al desarrollo de la tarea)

In [11]:
!pip install tinydb

In [13]:
df.sample()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
788,9.622106,182.909728,30307.217675,5.657004,422.445681,573.330497,17.699718,45.906782,3.975572,1


In [14]:
from datetime import datetime

# obtenemos un ejemplo de
ejemplo = df.sample()

# agregamos la fecha
hoy = datetime.now()
ejemplo["Day"] = hoy.day
ejemplo["Month"] = hoy.month
ejemplo["Year"] = hoy.year
ejemplo.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability,Day,Month,Year
1943,8.185923,240.979104,23641.472028,6.611449,318.68901,410.158222,13.109911,56.913981,3.566945,1,2,7,2022


In [15]:
ejemplo_dict = ejemplo.to_dict(orient="records")  # convertimos a diccionario
ejemplo_dict  # tinidb puede trabajar con este formato de datos

[{'ph': 8.185923417313038,
  'Hardness': 240.9791037623137,
  'Solids': 23641.472028426168,
  'Chloramines': 6.611448597083937,
  'Sulfate': 318.6890103719636,
  'Conductivity': 410.1582218451634,
  'Organic_carbon': 13.1099108733074,
  'Trihalomethanes': 56.91398086899888,
  'Turbidity': 3.566945145029043,
  'Potability': 1,
  'Day': 2,
  'Month': 7,
  'Year': 2022}]

Primero, creamos un cliente que cargue la base de datos (que ya contiene datos del agua).


In [106]:
from tinydb import Query, TinyDB

db = TinyDB("./db_ejemplo.json")

Para insertar el ejemplo en la base de datos, usamos `insert`

In [107]:
db.insert(
    ejemplo_dict[0]
)  # insertamos solo el diccionario, no la lista con el diccionario.

# el número que imprime (121) indica id (indice) interno del documento recién insertado.

129

Para pedir todos los datos almacenados, usamos `all()`

In [108]:
db.all()[0:10] # limitamos de 0 10 para que solo se muestren 10 ejemplos

[{'ph': 9.085637101693884,
  'Hardness': 238.22196802665175,
  'Solids': 9001.658753142132,
  'Chloramines': 9.820230918736057,
  'Sulfate': 325.1048470296242,
  'Conductivity': 381.3465821492757,
  'Organic_carbon': 19.19574006013147,
  'Trihalomethanes': 98.49081929088932,
  'Turbidity': 2.544020275544053,
  'Day': 15,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 6.592442507785291,
  'Hardness': 242.4804726770679,
  'Solids': 9381.239930798449,
  'Chloramines': 7.303183678175727,
  'Sulfate': 312.8143949567998,
  'Conductivity': 440.60113456028665,
  'Organic_carbon': 14.02729679512936,
  'Trihalomethanes': 70.92987947008055,
  'Turbidity': 3.0608273538358923,
  'Day': 15,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 8.990317233704655,
  'Hardness': 195.80902842150493,
  'Solids': 23728.29401456205,
  'Chloramines': 7.508305289508399,
  'Sulfate': 348.71461261339243,
  'Conductivity': 551.5403897934349,
  'Organic_carbon': 15.254391841066877,
  'Trihalometha

#### Consultas

Podemos hacer consultas, es decir, obtener algunos archivos según algún criterio, a través un objeto `Query` y `search`:

In [109]:
Mediciones = Query()

# Aquí Mediciones.Day representa a todas las llaves Day de todos los documentos.
# luego, el criterio de comparación es el que indica que datos se traeran.
# en la práctica, trae 40 datos, del día 14 al 15.
db.search(Mediciones.Day <= 15)

[{'ph': 9.085637101693884,
  'Hardness': 238.22196802665175,
  'Solids': 9001.658753142132,
  'Chloramines': 9.820230918736057,
  'Sulfate': 325.1048470296242,
  'Conductivity': 381.3465821492757,
  'Organic_carbon': 19.19574006013147,
  'Trihalomethanes': 98.49081929088932,
  'Turbidity': 2.544020275544053,
  'Day': 15,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 6.592442507785291,
  'Hardness': 242.4804726770679,
  'Solids': 9381.239930798449,
  'Chloramines': 7.303183678175727,
  'Sulfate': 312.8143949567998,
  'Conductivity': 440.60113456028665,
  'Organic_carbon': 14.02729679512936,
  'Trihalomethanes': 70.92987947008055,
  'Turbidity': 3.0608273538358923,
  'Day': 15,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 8.990317233704655,
  'Hardness': 195.80902842150493,
  'Solids': 23728.29401456205,
  'Chloramines': 7.508305289508399,
  'Sulfate': 348.71461261339243,
  'Conductivity': 551.5403897934349,
  'Organic_carbon': 15.254391841066877,
  'Trihalometha

Además, podemos mezclar distintos criterios de forma muy similar como lo hacemos en pandas:

En este caso, la siguiente consulta entrega todas las mediciones con ph entre 6 y 6.5 incluidas.

In [102]:
Mediciones = Query()

db.search((Mediciones.ph >= 6) & (Mediciones.ph <= 6.5))

[{'ph': 6.492627157365379,
  'Hardness': 213.06538481772856,
  'Solids': 34314.08709117726,
  'Chloramines': 9.04114608484344,
  'Sulfate': 335.29192590906365,
  'Conductivity': 313.2547429967247,
  'Organic_carbon': 11.135311827023884,
  'Trihalomethanes': 70.82910986892458,
  'Turbidity': 3.3787784927745594,
  'Day': 15,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 6.153368956728868,
  'Hardness': 237.13982778480292,
  'Solids': 22052.03461898169,
  'Chloramines': 8.090493963325427,
  'Sulfate': 306.23917986795703,
  'Conductivity': 485.0984831368037,
  'Organic_carbon': 17.07526258936946,
  'Trihalomethanes': 58.10825855736369,
  'Turbidity': 4.030337485422805,
  'Day': 17,
  'Month': 7,
  'Year': 2021,
  'Prediction': 0},
 {'ph': 6.470151959152821,
  'Hardness': 176.73637570597813,
  'Solids': 30594.28067473661,
  'Chloramines': 4.39313814981189,
  'Sulfate': 324.6592334189808,
  'Conductivity': 524.9097809286852,
  'Organic_carbon': 16.614247513308953,
  'Trihalometha

### Update

Podemos actualizar datos que hayan sido seleccionados usando una opración y una `Query` como argumentos de `update`.

Las operaciones disponibles son:

    delete(key): delete a key from the document
    increment(key): increment the value of a key
    decrement(key): decrement the value of a key
    add(key, value): add value to the value of a key (also works for strings)
    subtract(key, value): subtract value from the value of a key
    set(key, value): set key to value


In [110]:
from tinydb.operations import set

Mediciones = Query()

# en este caso, usaremos las mediciones del día 14 y fijaremos su ph a 7
db.update(set("ph", 7), Mediciones.Day == 14)
# noten que esta función retorna los indices de los documentos que fueron actualizados.

[]

In [105]:
db.search(Mediciones.Day == 14)

[]

### Eliminado


Por último, podemos eliminar de la BBDD con una `Query` y la función `remove`: 

In [28]:
db.remove(Mediciones.Day == 14)
# noten que esta función retorna los indices de los documentos que fueron borrados.

[]

In [29]:
db.search(Mediciones.Day == 14)

[]

### NOTA: Mutación de la BBDD y Respaldo

Cada vez que ejecutan insert, update o remove modifican las bases de datos. El archivo `db_resplado.json` contiene un respaldo de la base de datos para que puedan volver a utilizar la original.

-----

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<div align="center">
  <img src="https://i.pinimg.com/originals/84/5d/f1/845df1aefc6a5e37ae575327a0cc6e43.gif" width="500">
</div>